### 邻接矩阵的实现

In [17]:
class GraphError(Exception):
    pass

In [35]:
class Graph:
    def __init__(self, mat, unconn = 0): # unconn代表的是不能连接的边的大小
        vnum = len(mat) # 点的个数
        for x in mat:
            if len(x) != vnum:  # 检查是否为方阵
                raise ValueError("不是方阵！")
            self._mat = [mat[i][:] for i in range(vnum)] # 拷贝
            # 这里不用numpy的copy()可能也是这个原因。
            self._unconn = unconn
            self._vnum = vnum
    
    def vertex_num(self):
        return self._vnum

    def _invalid(self, v): # 判断点是不是合法
        return 0 > v or v >= self._vnum
    
    def add_edge(self, vi, vj, val = 1):
        if self._invalid(vi) or self._invalid(vj):
            raise GraphError(str(vi)+"或者"+str(vj)+"是非法点。")
            
    # 构造出边函数，即统计每个顶点的出边    
    def out_edges(self, vi):
        if self._invalid(vi):
            raise GraphError(str(vi) + "不是合法点")
        return self._out_edges(self._mat[vi], self._unconn)
    
    @staticmethod
    def _out_edges(row, unconn):
        edges = []
        for i in range(len(row)):
            if row[i] != unconn:
                edges.append((i, row[i]))
        return edges
    
    # str函数一般可以被print调用，如果a是一个Graph实例的话，
    # print(a)就会打印出__str__的内容
    def __str__(self):
        return "[\n" + ",\n".join(map(str, self._mat)) + "\n]" \
    + "\nUnconnected: " + str(self._unconn)

In [44]:
a = Graph([[0,4,1],[4,0,1],[1,1,0]])
# print(a)
a.out_edges(0)

[(1, 4), (2, 1)]

这个结果的意思是：0和1的链接权重为4，和2的链接权重为1

### 邻接表实现
邻接矩阵占用空间资源太多了，而且也不好加新点，就提出了邻接表的概念。邻接表被命名为GraphAL类，每个邻接边用一个（不一定等长的）list对象来表示。元素形式为$(v',w)$，$v'$是边的终点，$w$是边的权。

作者说，虽然GraphAL类的接口和Graph类的接口是一致的，但是内部数据结构存储的方式都变了，所以，其实可以不用继承Graph类。但是因为有一些函数功能重复，所以还是继承Graph类，然后对一些函数重新定义。

In [47]:
class GraphAL(Graph):
    def __init__(self, mat = [], unconn = 0):
        vnum = len(mat)
        for x in mat:
            if len(x) != vnum: # 检查是否为方阵
                raise ValueError("不是方阵！")
            # 把_mat属性转变为出边构成的列表
            self._mat = [Graph._out_edges(mat[i], unconn) 
                        for i in range(vnum)]
            
            self._vnum = vnum
            self._unconn = unconn
    
    # 加新点
    def add_vertex(self): # 增加新点时安排一个新编号
        self._mat.append([])
        self._vnum += 1
        return self._vnum - 1
    
    def add_edge(self, vi, vj, val = 1):
        if self._vnum == 0:
            raise GraphError("不能给空图加新边~")
        if self._invalid(vi) or self._invalid(vj):
            raise GraphError(str(vi) + "或者" + "str(vj)"+ "是非法点！")
            
        row = self._mat(vi)
        i = 0
        while i < len(row):
            if row[i][0] == vj: # 修改mat[vi][vj]的值(把矩阵形式改成邻接表)
                self._mat[vi][i] = (vj, val)
                return
            if row[i][0] > vj: # 这里的i是一个指针，指示的是vi已有的出边。
                break # 所有出边按顺序排列，所以一旦达到大于vj的指针，说明原来的边中并没有vj这条边，跳出循环
            i += 1
        # 在i指针位置上插入新的边
        self._mat[vi].insert(i, (vj, val))
    
    def get_edge(self, vi, vj):
        if self._invalid(vi) or self._invalid(vj):
            raise GraphError(str(vi) + "或者" + str(vj) + "是非法节点。")
        
        for i, val in self._mat(vi):
            if i == vj:
                return val
        return self._unconn
    
    def out_edges(self, vi):
        if self._invalid(vi):
            raise GraphError(str(vi) + "不是合法点")
        return self._mat[vi]

In [53]:
a = GraphAL([[0,4,1],[4,0,1],[1,1,0]])
a.out_edges(0)

[(1, 4), (2, 1)]